In [ ]:
# Set working drive
setwd("/Users/......................")

In [ ]:
library(dplyr)
library(lubridate)
library(readxl)
library(tidyr)
library(openxlsx)
library(tidy)

In [ ]:
# Read in Uber data
uber <- read_excel("Uber_peru_2010.xlsx", 3) 
uber.df = as.data.frame(uber)

In [ ]:
# Check out data 
str(uber.df)

In [ ]:
head(uber.df)

In [ ]:
# Eliminate unwanted columns
uber <- uber.df %>% select(-c(journey_id, user_id, driver_id, taxi_id, currency, ...29))

In [ ]:
# Check uber again
str(uber)

In [ ]:
# Extracting from start and end datetimes using lubridate
uber <- uber %>%
  mutate(
    Start_Yr = year(`start_at`),
    Start_Mth = month(`start_at`),
    Start_Hr = hour(`start_at`)
  )

uber <- uber %>%
  mutate(
    End_Yr = year(`end_at`),
    End_Mth = month(`end_at`),
    End_Hr = hour(`end_at`)
  )

In [ ]:
# Check uber data
str(uber)

In [ ]:
# Import in 2010 Temperature records
Temp <- read_excel("Peru 2010 Temp.xlsx", 1) 
Temp <- as.data.frame(Temp)

In [ ]:
# Check Temp
str(Temp)
head(Temp)

In [ ]:
# Tidy Temp 
Temp2 <- Temp %>% pivot_longer(c('Min', 'Average', 'Max'))%>% select(-Year) 

In [ ]:
# Check Temp data
head(Temp2)

In [ ]:
# Rename Temp columns
colnames(Temp2) <- c('Month', 'Time', 'Temp')

In [ ]:
# Check Temp data
head(Temp2)

In [ ]:
# Create variable for hours in Min, Max, Average in Uber
for (i in 1:nrow(uber)){
  if (between(uber$Start_Hr[i], 0, 7) | between(uber$Start_Hr[i], 22, 23)){
    uber$`Time`[i] <- 'Min'
  } else {
    if (between(uber$Start_Hr[i], 12, 17)){
      uber$`Time`[i] <- 'Max'
    } else {
      if (between(uber$Start_Hr[i], 8, 11) | between(uber$Start_Hr[i], 18, 21)){
        uber$`Time`[i] <- 'Average'
      } else {
        uber$`Time`[i] <- 'NA'      
      }
    }
  }
}

In [ ]:
# Check uber data
head(uber)

In [ ]:
# Lookup temperature in Temp2 table and merge into uber
M <- merge(uber, Temp2, by.x = c("Start_Mth", "Time"), by.y = c("Month", "Time"), all.x = TRUE, all.y = FALSE)

In [ ]:
# Check merged uber - Temp data
head(M)

In [ ]:
# Calculate travel time in seconds
M$Interval <- interval(M$`start_at`,M$`end_at`)
M$Travel <- int_length(M$Interval) # Interval in seconds

In [ ]:
# Check M
head(M)

In [ ]:
colSums(is.na(M))

In [ ]:
M <- M %>% filter(is.na(Travel) == F)

In [ ]:
# Check merged uber - Temp data
head(M)

In [ ]:
colSums(is.na(M))